Importing Libraries

In [2]:
import cv2
import time 

import tensorflow as tf
import os
import shutil
import numpy as np
import random
from tqdm import tqdm
from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
from pathlib import Path
import re
import cv2
from keras import backend as K
tf.__version__
import glob

#check if tensorflow gpu is being used
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("tensorflow version:", tf.__version__)

Num GPUs Available:  1
tensorflow version: 2.10.0


In [3]:
def stackImages(scale,imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range ( 0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape [:2]:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                else:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv2.cvtColor( imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv2.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None,scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor= np.hstack(imgArray)
        ver = hor
    return ver

In [4]:
def getContours(img,imgContour):
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        areaMin = cv2.getTrackbarPos("Area", "Parameters")
        if area > areaMin:
            cv2.drawContours(imgContour, cnt, -1, (255, 0, 255), 7)
            peri = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
            # print(len(approx))
            x , y , w, h = cv2.boundingRect(approx)
            cv2.rectangle(imgContour, (x , y ), (x + w , y + h ), (0, 255, 0), 5)

            # cv2.putText(imgContour, "Points: " + str(len(approx)), (x + w + 20, y + 20), cv2.FONT_HERSHEY_COMPLEX, .7,
            #             (0, 255, 0), 2)
            # cv2.putText(imgContour, "Area: " + str(int(area)), (x + w + 20, y + 45), cv2.FONT_HERSHEY_COMPLEX, 0.7,
            #             (0, 255, 0), 2)

In [12]:
#Realtime Segmentation with Dice Score and Bounding Box Detection

#Reference for bbox(Github): https://github.com/murtazahassan/OpenCV-Python-Tutorials-and-Projects/blob/master/Intermediate/RealTime_Shape_Detection_Contours.py 
#Reference for bbox (Video): https://www.youtube.com/watch?v=Fchzk1lDt7Q 

#Reference for realtime segmentation: https://anirudhtopiwala.com/my-robots/skin-segmentation-using-deep-learning/
#Github Code for realtime segmentation: https://github.com/anirudhtopiwala/abd-skin-segmentation/blob/master/Real%20Time%20Skin%20Segmentation/UNET-live.py 
#playing video on openCV: https://www.geeksforgeeks.org/python-play-a-video-using-opencv/ 
#adding FPS for openCV: https://www.geeksforgeeks.org/python-displaying-real-time-fps-at-which-webcam-video-file-is-processed-using-opencv/ 

#videos to choose from
NeedleViz_path1 = 'Data/edited data/102622_Water.mp4'
NeedleViz_path2 = 'Data/edited data/102822_Water.mp4'

#control playback speed
frame_rate = 25

# vc = cv2.VideoCapture(0) #opens camera
vc = cv2.VideoCapture(NeedleViz_path1)

prev_frame_time = 0
new_frame_time = 0
average_fps_list = []

frameWidth = 440
frameHeight = 440
vc.set(3, frameWidth)
vc.set(4, frameHeight)

def empty(a):
  pass

cv2.namedWindow("Parameters")
cv2.resizeWindow("Parameters",640,240)
cv2.createTrackbar("Threshold1","Parameters",155,255,empty)
cv2.createTrackbar("Threshold2","Parameters",191,255,empty)
cv2.createTrackbar("Threshold3","Parameters",0,13,empty)
cv2.createTrackbar("Threshold4","Parameters",0,13,empty)
cv2.createTrackbar("Area","Parameters",5000,30000,empty)


#Reference: https://stackoverflow.com/questions/39308030/how-do-i-increase-the-contrast-of-an-image-in-python-opencv
cv2.createTrackbar("alpha","Parameters", 255, 255, empty)
cv2.createTrackbar("beta","Parameters", 197, 255, empty)


if (vc.isOpened()== False): 
  print("Error opening video  file")

while(vc.isOpened()):
    rval, frame = vc.read()
    
    if rval == True:
      #Insert Code Here

      new_frame_time = time.time()
      fps = 1/(new_frame_time-prev_frame_time)
      prev_frame_time = new_frame_time

      fps = int(fps)
      average_fps_list.append(fps)

      ## Initial Image Processing ##
  
      raw_image_1 = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
      raw_image = cv2.resize(raw_image_1, (frameWidth,frameHeight))
      # raw_image = raw_image_1
      # print(raw_image.shape)
      #Converting float64 to uint8 for proper processing in opencv
      updated_maskV2 = raw_image.astype(np.uint8)
      imgContour = raw_image.copy() #copy desired frame (normal frame or predicted mask) to have contour and bboxes layed over for realtime display
      

      #smoothens the image 
      imgBlur = cv2.GaussianBlur(updated_maskV2,(7,7), 1)


      #Adjusting contrast levels (issue was that the original image was so dim it could not be seen as mentioned during a warning)
      alpha = cv2.getTrackbarPos("alpha", "Parameters")
      beta = cv2.getTrackbarPos("beta", "Parameters")
      contrast_imageBlur = alpha*imgBlur+beta

      #detects canny edges
      threshold1 = cv2.getTrackbarPos("Threshold1", "Parameters")
      threshold2 = cv2.getTrackbarPos("Threshold2", "Parameters")
      imgCanny_initial = cv2.Canny(contrast_imageBlur, threshold1, threshold2)

      #Detect vertical lines (reference: https://www.youtube.com/watch?v=veoz_46gOkc&ab_channel=ProgrammingEpitome)
      threshold3 = cv2.getTrackbarPos("Threshold3", "Parameters")
      threshold4 = cv2.getTrackbarPos("Threshold4", "Parameters")
      kernel_vertical = np.ones((threshold3,threshold4), np.uint8)
      imgCanny = cv2.erode(imgCanny_initial, kernel_vertical, iterations=1)


      #dilation to further remove any noise generated by canny detection 
      kernel = np.ones((5, 5))
      imgDil = cv2.dilate(imgCanny_initial, kernel, iterations=1)
      getContours(imgDil,imgContour)
      
      
      # putting the FPS count and dice score evalution on the preview frame
      cv2.putText(imgCanny_initial, 'FPS: {}'.format(str(fps)), (7, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
      # cv2.putText(imgCanny, 'DiceScore: {}'.format(str(realtime_dice)), (180, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
      

      # SEPERATE WINDOWS #
      # cv2.imshow("preview1", contrast_imageBlur) #originally preds_resized
      # cv2.imshow("preview2", imgCanny) #originally preds_resized
      # cv2.imshow("preview3", imgContour)
      # cv2.imshow("normal",raw_image)
      
      # COMBINED WINDOWS #
      imgStack = stackImages(0.8,([raw_image,contrast_imageBlur],[imgCanny_initial,imgContour]))
      cv2.imshow("stacked", imgStack)


      # Press Q on keyboard to  exit
      if cv2.waitKey(frame_rate) & 0xFF == ord('q'): #original waitkey is 25
          break
    
    #Break out of loop if video is done
    else:
        break  

vc.release() #Release the video capture object

# Close windows

# cv2.destroyWindow("normal")
# cv2.destroyWindow("preview1")
# cv2.destroyWindow("preview2")
# cv2.destroyWindow("preview3")

cv2.destroyWindow("stacked")


cv2.destroyWindow("Parameters")
